<a href="https://colab.research.google.com/github/QuinnStraus/CSCI-1470-final/blob/main/DL_Final_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>